# Jupyter client 

`jupyter_client` is a package that implements a system for managing jupyter kernels. See the [official documentation](https://jupyter-client.readthedocs.io/en/latest/index.html). Actually, it provides access to the programming management of jupyter.

**Note:** Run this notebook in the same environment, as it intentionally modifies the environment to demonstrate certain configurations.

## CLI

`jupyter-kernel` tool comes with `jupyter_client` package.

---

It confuses me that `jupyter_client` provides the `jupyter-kernel` CLI tool. If you have the same doubts, the following cells will prove it.

The code in the following cell shows the file to which `jupyter-kernel` refers in the system.

In [1]:
!which jupyter-kernel

/usr/local/bin/jupyter-kernel


And behind it is Python code that refers to `jupyter_client.kernelapp.main`.

In [2]:
!cat $(which jupyter-kernel)

#!/usr/local/bin/python3.12
# -*- coding: utf-8 -*-
import re
import sys
from jupyter_client.kernelapp import main
if __name__ == '__main__':
    sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])
    sys.exit(main())


## Run from python

Sometimes it's useful to be able to run the kernel from the python code - for debugging/experimenting purposes. The main tool for such purposes is the `ipykernel.kernelbase.Kernel` class, which allows to run the kernel and automatically launch the client to it.

- Check some details in [KernelManager documentation page](https://jupyter-client.readthedocs.io/en/latest/api/jupyter_client.html#jupyter_client.manager.KernelManager).
- And [specific page](jupyter_client/running_kernel.ipynb) in this web-site.

---

The following example starts the Python kernel.

In [7]:
from jupyter_client import KernelManager
km = KernelManager(kernel_name="python3")
km.start_kernel()

kc = km.client()
kc.start_channels()

Kernel is a separate process, to which you can send code you want to execute and wait for the response. The following cell makes exactly that.

In [8]:
kc.execute("print(f'hello from kernel {10 + 99}')", reply=True)
msg = kc.get_iopub_msg()
while True:
   msg = kc.get_iopub_msg(timeout=5)
   if msg['msg_type'] == 'stream':
        break

Once we have received the response, we can check it.

In [9]:
print(msg["content"]["text"])

hello from kernel 109



Expected output for the code we sent to kernel to execute.

Kernel must be shut down:

In [10]:
kc.shutdown(reply=True)
km.shutdown_kernel(now=True)

## Creating kernel

There are three ways to create your own Jupyter kernel; check more [here](https://jupyter-client.readthedocs.io/en/latest/kernels.html). This page focuses on the simplest method: [simple Python wrapper kernels](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html).

For creating wrapper kernel you need:

- Write a file that implements the logic of the kernel. The official approach is to inherit from the `ipykernel.kernelbase.Kernel` class.  
- Register and set the kernel's configuration to refer to the kernel implementation.

---

The following cell shows the implementation of the kernel.

In [1]:
%%writefile /tmp/my_kernel.py
from ipykernel.kernelbase import Kernel

class EchoKernel(Kernel):
    implementation = 'Echo'
    implementation_version = '1.0'
    language = 'no-op'
    language_version = '0.1'
    language_info = {
        'name': 'Any text',
        'mimetype': 'text/plain',
        'file_extension': '.txt',
    }
    banner = "Echo kernel - as useful as a parrot"

    def do_execute(
        self,
        code,
        silent,
        store_history=True,
        user_expressions=None,
        allow_stdin=False
    ):
        if not silent:
            stream_content = {'name': 'stdout', 'text': code}
            self.send_response(self.iopub_socket, 'stream', stream_content)

        return {
            'status': 'ok',
            'execution_count': self.execution_count,
            'payload': [],
            'user_expressions': {},
        }

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(kernel_class=EchoKernel)

Overwriting /tmp/my_kernel.py


The next step is to register the kernel, which involves creating a `json` configuration file that points to the Python code you created earlier. First, we should check where existing kernels are stored using the `jupyter kernelspec list` command.

In [14]:
!jupyter kernelspec list

Available kernels:
  python3    /usr/local/share/jupyter/kernels/python3


At the same path, we need to create a folder for the kernel and place `kernel.json` inside it. The `argv` key in `kernel.json` should execute the module we created earlier.

In [3]:
!mkdir -p /usr/local/share/jupyter/kernels/echo

In [4]:
%%writefile /usr/local/share/jupyter/kernels/echo/kernel.json
{
    "argv":[
        "python3", 
        "/tmp/my_kernel.py", 
        "-f", 
        "{connection_file}"
    ],
    "display_name":"Echo"
}

Overwriting /usr/local/share/jupyter/kernels/echo/kernel.json


Now the result of the `jupyter kernelspec list` command is the kernel you just created.

In [5]:
!jupyter kernelspec list

Available kernels:
  echo       /usr/local/share/jupyter/kernels/echo
  python3    /usr/local/share/jupyter/kernels/python3


To ensure that everything works correct let's try to run `echo` kernel and send `hello, parrot?` to be executed.

In [11]:
from jupyter_client import KernelManager
km = KernelManager(kernel_name="echo")
km.start_kernel()

kc = km.client()
kc.start_channels()

kc.execute("hello, parrot?", reply=True)
msg = kc.get_iopub_msg()
while True:
   msg = kc.get_iopub_msg(timeout=5)
   if msg['msg_type'] == 'stream':
        break

kc.shutdown(reply=True)
km.shutdown_kernel(now=True)

As a result, the kernel returns exactly what we've sent it - exactly as specified in its logic.

In [12]:
msg["content"]["text"]

'hello, parrot?'

Even better, you can run a Jupyter Notebook file, select `echo` from the list of kernels, and use it just like any other notebook.